In [16]:
from ts_config import GenerationConfig
import xarray
from distributed import Client
from dask.distributed import wait, TimeoutError
from uuid import uuid4
from shutil import rmtree
from pathlib import Path

client = Client('localhost:8786')
client

<Client: 'tcp://146.6.192.149:8786' processes=10 threads=10, memory=465.66 GiB>

In [13]:
config = GenerationConfig(["/local1/testing_timeseries/history_files/v21.LR.BSMYLEsmbb.1970-05.001/archive.001/"], "/local1/testing_timeseries/output/")
interm_sizes = config.fit_interm_timeseries_to_memory()
batches = config.get_timeseries_batches(interm_sizes)
batches[0][0]

Sampling dataset metadata from each case group to estimate total size in memory (this may take some time)...


PosixPath('/local1/testing_timeseries/output/archive.001/atm/tseries')

In [18]:
%%time
for output_dir, group, batch_paths in batches:
    print(f"Post-processing {group} ... ", end="")
    output_dir.mkdir(parents=True, exist_ok=True)
    history_zarr_path = f"{output_dir}/tmp_hs_store.zarr"

    history_concat = xarray.open_mfdataset(batch_paths, parallel=True).chunk(dict(time=1))
    history_concat.to_zarr(history_zarr_path, mode="w", consolidated=True)

    dt = history_concat.time.values[1] - history_concat.time.values[0]
    if dt.days == 0:
        time_str_format = "%Y-%m-%d-%H"
    elif 30 > dt.days > 0:
        time_str_format = "%Y-%m-%d"
    else:
        time_str_format = "%Y-%m"
    
    time_start = history_concat.time.values[0].strftime(time_str_format)
    time_end = history_concat.time.values[-1].strftime(time_str_format)
    
    attribute_variables = []
    for variable in list(history_concat.variables):
        if "cell_methods" not in history_concat[variable].attrs:
            attribute_variables.append(variable)
    
    def export_dataset(config_tuple):
        zarr_path, variables, output_path, uid = config_tuple
        xarray.open_zarr(zarr_path)[variables].to_netcdf(output_path, mode="w")
        return uid
    
    config_tuples = [(history_zarr_path,
                      [variable],
                      f"{output_dir}/{group}.{variable}.{time_start}.{time_end}.nc",
                      uuid4())
                     for variable in list(history_concat.variables) if variable not in attribute_variables]
    
    futures = client.map(export_dataset, config_tuples)

    attrs_ds = xarray.open_zarr(history_zarr_path)[attribute_variables].compute()
    
    for task in futures:
        wait(task)
        task.release()
    client.amm.stop()
    scatted_attrs = client.scatter(attrs_ds, broadcast=True)
    
    def add_descriptive_variables(path_ds_tuple):
        ds, path = path_ds_tuple
        ds.to_netcdf(path, mode="a")
    
    path_tuples = [(scatted_attrs, f"{output_dir}/{group}.{variable}.{time_start}.{time_end}.nc") for variable in list(history_concat.variables) if variable not in attribute_variables]
    futures = client.map(add_descriptive_variables, path_tuples)
    
    for task in futures:
        wait(task)
        task.release()
    scatted_attrs.release()
    
    rmtree(history_zarr_path)
    client.amm.start()
    print("done!")
    break

Post-processing v21.LR.BSMYLEsmbb.1970-05.001.eam.h1 ... 

/home/jupyterhub/miniconda-persad/envs/lab/lib/python3.12/site-packages/distributed/client.py:3164: UserWarning: Sending large graph of size 17.13 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


done!
CPU times: user 29.9 s, sys: 7.4 s, total: 37.3 s
Wall time: 5min 42s
